In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import altair as alt

from matplotlib import pyplot as plt
plt.style.use('seaborn-v0_8')

import gc

import logging
import warnings

warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Fall_2025/CPSC_5305_Intro_to_DS/data/"

In [4]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_datetime64_any_dtype(col_type):
            continue

        # only process numeric columns for min/max
        if col_type != object and not isinstance(col_type, pd.CategoricalDtype):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        # handle categorical columns without min/max
        elif isinstance(col_type, pd.CategoricalDtype):
            df[col] = df[col].cat.as_unordered()  # ensure no ordering assumption
        else:
            df[col] = df[col].astype('category')  # convert objects to category
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Memory usage decreased from {start_mem:.2f} Mb to {end_mem:.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df

In [5]:
print("Loading and reducing memory usage for data files...")

# load the datasets and apply memory reduction
calendar = pd.read_csv(f'{DATA_PATH}calendar.csv')
calendar = reduce_mem_usage(calendar)

sell_prices = pd.read_csv(f'{DATA_PATH}sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)

# using 'sales_train_evaluation.csv' for the full data because it's the most complete one
sales = pd.read_csv(f'{DATA_PATH}sales_train_evaluation.csv')
sales = reduce_mem_usage(sales)

print("\nInitial data loading complete.")

Loading and reducing memory usage for data files...
Memory usage decreased from 0.21 Mb to 0.19 Mb (8.7% reduction)
Memory usage decreased from 208.77 Mb to 58.80 Mb (71.8% reduction)
Memory usage decreased from 452.91 Mb to 96.30 Mb (78.7% reduction)

Initial data loading complete.


## Calendar

In [6]:
print(calendar.head().to_markdown())

|    | date       |   wm_yr_wk | weekday   |   wday |   month |   year | d   |   event_name_1 |   event_type_1 |   event_name_2 |   event_type_2 |   snap_CA |   snap_TX |   snap_WI |
|---:|:-----------|-----------:|:----------|-------:|--------:|-------:|:----|---------------:|---------------:|---------------:|---------------:|----------:|----------:|----------:|
|  0 | 2011-01-29 |      11101 | Saturday  |      1 |       1 |   2011 | d_1 |            nan |            nan |            nan |            nan |         0 |         0 |         0 |
|  1 | 2011-01-30 |      11101 | Sunday    |      2 |       1 |   2011 | d_2 |            nan |            nan |            nan |            nan |         0 |         0 |         0 |
|  2 | 2011-01-31 |      11101 | Monday    |      3 |       1 |   2011 | d_3 |            nan |            nan |            nan |            nan |         0 |         0 |         0 |
|  3 | 2011-02-01 |      11101 | Tuesday   |      4 |       2 |   2011 | d_4 |       

In [7]:
calendar['date'] = pd.to_datetime(calendar['date'])

In [8]:
print(calendar.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int16         
 2   weekday       1969 non-null   category      
 3   wday          1969 non-null   int8          
 4   month         1969 non-null   int8          
 5   year          1969 non-null   int16         
 6   d             1969 non-null   category      
 7   event_name_1  162 non-null    category      
 8   event_type_1  162 non-null    category      
 9   event_name_2  5 non-null      category      
 10  event_type_2  5 non-null      category      
 11  snap_CA       1969 non-null   int8          
 12  snap_TX       1969 non-null   int8          
 13  snap_WI       1969 non-null   int8          
dtypes: category(6), datetime64[ns](1), int16(2), int8(5)
memory usage: 128.3 KB
None


In [9]:
print(calendar.shape)

(1969, 14)


In [10]:
print(calendar.isnull().sum().to_markdown())

|              |    0 |
|:-------------|-----:|
| date         |    0 |
| wm_yr_wk     |    0 |
| weekday      |    0 |
| wday         |    0 |
| month        |    0 |
| year         |    0 |
| d            |    0 |
| event_name_1 | 1807 |
| event_type_1 | 1807 |
| event_name_2 | 1964 |
| event_type_2 | 1964 |
| snap_CA      |    0 |
| snap_TX      |    0 |
| snap_WI      |    0 |


In [11]:
# Data cleaning & transformation for calendar

# fill missing event values
event_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

# convert to object, fill, then convert to category, this handles potential Categorical dtype errors and cleans the NaNs.
calendar[event_cols] = calendar[event_cols].astype('object')
calendar[event_cols] = calendar[event_cols].fillna('No event')
calendar[event_cols] = calendar[event_cols].astype('category')

calendar['date'] = pd.to_datetime(calendar['date'])

# verify missing values (should be all 0 now)
print("Missing values in calendar after cleaning:")
print(calendar.isnull().sum().to_markdown())

# data reduction: convert categorical columns to the category dtype
for col in calendar.columns:
    if calendar[col].dtype == 'object':
        calendar[col] = calendar[col].astype('category')

print("\nCalendar data cleaned and transformed.")

print(calendar.head().to_markdown())

Missing values in calendar after cleaning:
|              |   0 |
|:-------------|----:|
| date         |   0 |
| wm_yr_wk     |   0 |
| weekday      |   0 |
| wday         |   0 |
| month        |   0 |
| year         |   0 |
| d            |   0 |
| event_name_1 |   0 |
| event_type_1 |   0 |
| event_name_2 |   0 |
| event_type_2 |   0 |
| snap_CA      |   0 |
| snap_TX      |   0 |
| snap_WI      |   0 |

Calendar data cleaned and transformed.
|    | date                |   wm_yr_wk | weekday   |   wday |   month |   year | d   | event_name_1   | event_type_1   | event_name_2   | event_type_2   |   snap_CA |   snap_TX |   snap_WI |
|---:|:--------------------|-----------:|:----------|-------:|--------:|-------:|:----|:---------------|:---------------|:---------------|:---------------|----------:|----------:|----------:|
|  0 | 2011-01-29 00:00:00 |      11101 | Saturday  |      1 |       1 |   2011 | d_1 | No event       | No event       | No event       | No event       |         0

## Sales

In [12]:
print(sales.head().to_markdown())

|    | id                            | item_id       | dept_id   | cat_id   | store_id   | state_id   |   d_1 |   d_2 |   d_3 |   d_4 |   d_5 |   d_6 |   d_7 |   d_8 |   d_9 |   d_10 |   d_11 |   d_12 |   d_13 |   d_14 |   d_15 |   d_16 |   d_17 |   d_18 |   d_19 |   d_20 |   d_21 |   d_22 |   d_23 |   d_24 |   d_25 |   d_26 |   d_27 |   d_28 |   d_29 |   d_30 |   d_31 |   d_32 |   d_33 |   d_34 |   d_35 |   d_36 |   d_37 |   d_38 |   d_39 |   d_40 |   d_41 |   d_42 |   d_43 |   d_44 |   d_45 |   d_46 |   d_47 |   d_48 |   d_49 |   d_50 |   d_51 |   d_52 |   d_53 |   d_54 |   d_55 |   d_56 |   d_57 |   d_58 |   d_59 |   d_60 |   d_61 |   d_62 |   d_63 |   d_64 |   d_65 |   d_66 |   d_67 |   d_68 |   d_69 |   d_70 |   d_71 |   d_72 |   d_73 |   d_74 |   d_75 |   d_76 |   d_77 |   d_78 |   d_79 |   d_80 |   d_81 |   d_82 |   d_83 |   d_84 |   d_85 |   d_86 |   d_87 |   d_88 |   d_89 |   d_90 |   d_91 |   d_92 |   d_93 |   d_94 |   d_95 |   d_96 |   d_97 |   d_98 |   d_99 |   d_100 |   d_

In [13]:
print(sales.isnull().sum())

id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
           ..
d_1937      0
d_1938      0
d_1939      0
d_1940      0
d_1941      0
Length: 1947, dtype: int64


In [14]:
print(sales.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: category(6), int16(1317), int8(624)
memory usage: 96.3 MB
None


In [15]:
print(sales.shape)

(30490, 1947)


In [16]:
# show only columns with missing (NaN) values
sales.isnull().sum()[sales.isnull().sum() > 0]

,0


In [17]:
# Data transformation for 'sales' (Melting to Long Format)

# identify the columns to keep as identifiers
id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# identify the sale columns (d_1 to d_1941)
sales_cols = [col for col in sales.columns if col.startswith('d_')]

print("Melting sales data...")

# melt the df
sales_melted = sales.melt(
    id_vars=id_vars,
    value_vars=sales_cols,
    var_name='d',  # new column for the day ID (d_1, d_2, etc.)
    value_name='sales' # new column for the actual sales volume
)

sales_melted = reduce_mem_usage(sales_melted)


print("Sales data melted and memory reclaimed.")
print(f"Melted sales shape: {sales_melted.shape}")

Melting sales data...
Memory usage decreased from 1017.24 Mb to 678.68 Mb (33.3% reduction)
Sales data melted and memory reclaimed.
Melted sales shape: (59181090, 8)


In [18]:
print(sales_melted.head().to_markdown())

|    | id                            | item_id       | dept_id   | cat_id   | store_id   | state_id   | d   |   sales |
|---:|:------------------------------|:--------------|:----------|:---------|:-----------|:-----------|:----|--------:|
|  0 | HOBBIES_1_001_CA_1_evaluation | HOBBIES_1_001 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |
|  1 | HOBBIES_1_002_CA_1_evaluation | HOBBIES_1_002 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |
|  2 | HOBBIES_1_003_CA_1_evaluation | HOBBIES_1_003 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |
|  3 | HOBBIES_1_004_CA_1_evaluation | HOBBIES_1_004 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |
|  4 | HOBBIES_1_005_CA_1_evaluation | HOBBIES_1_005 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |


In [19]:
print(sales_melted.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 8 columns):
 #   Column    Dtype   
---  ------    -----   
 0   id        category
 1   item_id   category
 2   dept_id   category
 3   cat_id    category
 4   store_id  category
 5   state_id  category
 6   d         category
 7   sales     int16   
dtypes: category(7), int16(1)
memory usage: 678.7 MB
None


In [20]:
print(sales_melted.isnull().sum())

id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
state_id    0
d           0
sales       0
dtype: int64


In [21]:
print(sales_melted['sales'].describe())

count    5.918109e+07
mean     1.130888e+00
std      3.870038e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      7.630000e+02
Name: sales, dtype: float64


## Sell prices

In [22]:
print(sell_prices.head().to_markdown())

|    | store_id   | item_id       |   wm_yr_wk |   sell_price |
|---:|:-----------|:--------------|-----------:|-------------:|
|  0 | CA_1       | HOBBIES_1_001 |      11325 |         9.58 |
|  1 | CA_1       | HOBBIES_1_001 |      11326 |         9.58 |
|  2 | CA_1       | HOBBIES_1_001 |      11327 |         8.26 |
|  3 | CA_1       | HOBBIES_1_001 |      11328 |         8.26 |
|  4 | CA_1       | HOBBIES_1_001 |      11329 |         8.26 |


In [23]:
print(sell_prices.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype   
---  ------      -----   
 0   store_id    category
 1   item_id     category
 2   wm_yr_wk    int16   
 3   sell_price  float32 
dtypes: category(2), float32(1), int16(1)
memory usage: 58.8 MB
None


In [24]:
print(sell_prices.shape)

(6841121, 4)


In [25]:
print(sell_prices.isnull().sum())

store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64


In [26]:
print(sell_prices['sell_price'].describe())

count    6.841121e+06
mean     4.410955e+00
std      3.390167e+00
min      1.000000e-02
25%      2.180000e+00
50%      3.470000e+00
75%      5.840000e+00
max      1.073200e+02
Name: sell_price, dtype: float64


## Final dataframe

In [27]:
# merge with calendar
sales_with_cal_and_wm = sales_melted.merge(
    calendar[['d', 'wm_yr_wk']],
    on='d',
    how='left'
)

# merge with sell_prices
df = sales_with_cal_and_wm.merge(
    sell_prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

print(df.head().to_markdown())

|    | id                            | item_id       | dept_id   | cat_id   | store_id   | state_id   | d   |   sales |   wm_yr_wk |   sell_price |
|---:|:------------------------------|:--------------|:----------|:---------|:-----------|:-----------|:----|--------:|-----------:|-------------:|
|  0 | HOBBIES_1_001_CA_1_evaluation | HOBBIES_1_001 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |      11101 |          nan |
|  1 | HOBBIES_1_002_CA_1_evaluation | HOBBIES_1_002 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |      11101 |          nan |
|  2 | HOBBIES_1_003_CA_1_evaluation | HOBBIES_1_003 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |      11101 |          nan |
|  3 | HOBBIES_1_004_CA_1_evaluation | HOBBIES_1_004 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |      11101 |          nan |
|  4 | HOBBIES_1_005_CA_1_evaluation | HOBBIES_1_005 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |   

In [28]:
# merge calendar features again (since wm_yr_wk merge was intermediary)
df = df.merge(
    calendar[['d', 'date', 'weekday', 'wday', 'month', 'year', 'snap_CA', 'snap_TX', 'snap_WI', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']],
    on='d',
    how='left',
    suffixes=('_sales', '_cal')
)

In [29]:
print(df.head().to_markdown())

|    | id                            | item_id       | dept_id   | cat_id   | store_id   | state_id   | d   |   sales |   wm_yr_wk |   sell_price | date                | weekday   |   wday |   month |   year |   snap_CA |   snap_TX |   snap_WI | event_name_1   | event_type_1   | event_name_2   | event_type_2   |
|---:|:------------------------------|:--------------|:----------|:---------|:-----------|:-----------|:----|--------:|-----------:|-------------:|:--------------------|:----------|-------:|--------:|-------:|----------:|----------:|----------:|:---------------|:---------------|:---------------|:---------------|
|  0 | HOBBIES_1_001_CA_1_evaluation | HOBBIES_1_001 | HOBBIES_1 | HOBBIES  | CA_1       | CA         | d_1 |       0 |      11101 |          nan | 2011-01-29 00:00:00 | Saturday  |      1 |       1 |   2011 |         0 |         0 |         0 | No event       | No event       | No event       | No event       |
|  1 | HOBBIES_1_002_CA_1_evaluation | HOBBIES_1_002 | HOB

In [30]:
print(df.isna().sum())

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
d                      0
sales                  0
wm_yr_wk               0
sell_price      12299413
date                   0
weekday                0
wday                   0
month                  0
year                   0
snap_CA                0
snap_TX                0
snap_WI                0
event_name_1           0
event_type_1           0
event_name_2           0
event_type_2           0
dtype: int64


In [31]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            category      
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             object        
 7   sales         int16         
 8   wm_yr_wk      int16         
 9   sell_price    float32       
 10  date          datetime64[ns]
 11  weekday       category      
 12  wday          int8          
 13  month         int8          
 14  year          int16         
 15  snap_CA       int8          
 16  snap_TX       int8          
 17  snap_WI       int8          
 18  event_name_1  category      
 19  event_type_1  category      
 20  event_name_2  category      
 21  event_type_2  category      
dtypes: category(11), datetime64[ns](1), float32(1), int16(3), int8(5

## Preprocessing

sell_price has missing values. These represent days when the product was not sold (likely not in stock or not yet released). A common strategy is to forward-fill the prices, assuming the price remains the same from the last day it was sold.

In [32]:
# sort by id and date to ensure correct forward fill
df.sort_values(by=['id', 'date'], inplace=True)

# group by 'id' and forward-fill missing prices
# this ensures we don't bleed prices from one item-store to another
df['sell_price'] = df.groupby('id')['sell_price'].ffill()

# for any remaining NaNs at the beginning (before first sale), we can backfill
df['sell_price'] = df.groupby('id')['sell_price'].bfill()

# NOTE:
# When the code runs df.groupby('id')['sell_price'].ffill(),
# it is grouping by that entire unique ID -> so, it's already creating
# a separate group for each item at each specific store.
# For example, it will process:
# HOBBIES_1_001_CA_1 as one group.
# HOBBIES_1_001_CA_2 as a completely separate group.
# HOBBIES_1_001_TX_1 as yet another separate group.
# ....
# -> SO: no worry about filling same price for items between states

# Because of this, the ffill() (forward fill) and bfill() (backward fill)
# operations only use prices from within that same item's history at that same store.
# It will not use a price from store CA_1 to fill a missing value for the same item in store TX_1.

print("Missing values after processing:")
print(df.isnull().sum())

Missing values after processing:
id              0
item_id         0
dept_id         0
cat_id          0
store_id        0
state_id        0
d               0
sales           0
wm_yr_wk        0
sell_price      0
date            0
weekday         0
wday            0
month           0
year            0
snap_CA         0
snap_TX         0
snap_WI         0
event_name_1    0
event_type_1    0
event_name_2    0
event_type_2    0
dtype: int64


In [33]:
gc.collect()

0

In [34]:
df.to_parquet(DATA_PATH + "final_m5_processed.parquet", index=False)